In [ ]:
import numpy as np
import pandas as pd
import time
import warnings
import csv
import datetime
from datetime import datetime, date, time, timedelta

# Now let's bring in some new libraries from scipy
from scipy import stats

In [ ]:
#Uploading of the event logs of the 5 subprocesses as DataFrame
parse_dates = ['completeTime']
df_CC_CM = pd.read_csv ('CC_compliantManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
df_CC_OM = pd.read_csv ('CC_orderManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
onlyTS = pd.read_csv ('CC_orderManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
df_FA_MI = pd.read_csv ('FA_manageInstallation.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
df_SA_PM = pd.read_csv ('SA_paymentManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)
df_SA_SM = pd.read_csv ('SA_serviceManagement.csv', sep=",", header=0,engine="python", quoting=0, parse_dates=parse_dates)

In [ ]:
#For each event log, the useless columns are dropped to fasten their management
df_CC_CM = df_CC_CM.drop('simulated:logProbability', axis=1)
df_CC_CM = df_CC_CM.drop('concept:simulated', axis=1)
df_CC_CM = df_CC_CM.drop('startTime', axis=1)
df_CC_CM = df_CC_CM.drop('completeTime', axis=1)

df_CC_OM = df_CC_OM.drop('concept:simulated', axis=1)
df_CC_OM = df_CC_OM.drop('simulated:logProbability', axis=1)
df_CC_OM = df_CC_OM.drop('startTime', axis=1)
df_CC_OM = df_CC_OM.drop('completeTime', axis=1)

df_FA_MI = df_FA_MI.drop('simulated:logProbability', axis=1)
df_FA_MI = df_FA_MI.drop('startTime', axis=1)
df_FA_MI = df_FA_MI.drop('concept:simulated', axis=1)
df_FA_MI = df_FA_MI.drop('completeTime', axis=1)

df_SA_PM = df_SA_PM.drop('startTime', axis=1)
df_SA_PM = df_SA_PM.drop('simulated:logProbability', axis=1)
df_SA_PM = df_SA_PM.drop('concept:simulated', axis=1)
df_SA_PM = df_SA_PM.drop('completeTime', axis=1)

df_SA_SM = df_SA_SM.drop('simulated:logProbability', axis=1)
df_SA_SM = df_SA_SM.drop('concept:simulated', axis=1)
df_SA_SM = df_SA_SM.drop('startTime', axis=1)
df_SA_SM = df_SA_SM.drop('completeTime', axis=1)

In [ ]:
final_df = pd.DataFrame(columns = ['case','event','time','fuelUsed','paymentTerms','status']) # The columns needed in the final event logs are added.
logs = range(5000)

In [ ]:
for i in logs:
     
    j = np.random.randint(1, 10)
    if(((df_CC_OM['event'] == 'end_event_notification unavailability') & (df_CC_OM['case'] == i)).any() == False): #based on the types of events found in the subprocesses' logs, the istance is built
        ts = onlyTS.iloc[0]['completeTime'] + timedelta(minutes = j) # the timestamp of each first activity of each instance is randomly created
        d1 = pd.DataFrame()

        # CC_ORDER_MANAGEMENT
        d1 = df_CC_OM.head(7) # select the rows that have to be moved from the subprocess' log to the final one
        n = np.random.randint(1, 10) #random integer used to inject the randomness of the timestamps
        d1.insert(2,"time",[ts, ts + timedelta(minutes = 1), ts + timedelta(minutes = 2), ts + timedelta(minutes = 10), ts + timedelta(minutes =12), ts + timedelta(hours = n),ts + timedelta(hours = n+1)]) # creation of the timestamps of each event that has to be move
        final_df = pd.concat([final_df, d1.head(7)]) # physical move of the tuples to the final event log
        df_CC_OM.drop(df_CC_OM.index[:7], inplace=True) # drop of the tuples copied in the final event log
        
        # the procedure explained above is repeated each time, for each subprocess until the process istance is correctly built

        #SA_SERVICE_MANAGEMENT
        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_SA_SM.head(6)
        d1.insert(2,"time",[ts, ts + timedelta(minutes = 1), ts + timedelta(minutes = 25), ts + timedelta(minutes = 26), ts + timedelta(minutes =26), ts + timedelta(minutes = 28)])
        
        d1.insert(3,"status",['','','activated', '', '','']) #the attribute is injected only in the appropriate event, in the other tuples the attribute is a '' that will be interpreted as NULL
        final_df = pd.concat([final_df, d1.head(6)]) #until evaluateOrderParameters
        df_SA_SM.drop(df_SA_SM.index[:6], inplace=True) 
       
        #FA_MANAGE_INSTALLATION
        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_FA_MI.head(3)
        d1.insert(2,"time",[ts, ts + timedelta(minutes = 1), ts + timedelta(minutes = 15)])
        final_df = pd.concat([final_df, d1.head(3)])
        df_FA_MI.drop(df_FA_MI.index[:3], inplace=True)

        #CC_COMPLIANT_MANAGEMENT
        if(((df_SA_SM['event'] == 'solicitation from the call center') & (df_SA_SM['case'] == i)).any() == True): # select the rows that have to be moved from the subprocess' log to the final one
            ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
            d1 = pd.DataFrame()
            d1 = df_CC_CM.head(4)
            n = np.random.randint(1, 24)
            start = ts
            if(((df_CC_CM['event'] == 'end_event_keep calm') & (df_CC_CM['case'] == i)).any() == False): 
                start = ts + timedelta(days = 7)
                ts = start

            d1.insert(2,"time",[start, ts + timedelta(minutes = 3),ts + timedelta(minutes = 30) , ts + timedelta(hours = n)])     
            final_df = pd.concat([final_df, d1.head(4) ])
            df_CC_CM.drop(df_CC_CM.index[:4], inplace=True)
            
        #SA_SERVICE_MANAGEMENT
            if(((final_df['event'] == 'end_event_keep calm') & (final_df['case'] == i)).any() == False): 
                ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
                d1 = pd.DataFrame()
                d1 = df_SA_SM.head(2)
                n = np.random.randint(1, 24)
                d1.insert(2,"time",[ts, ts + timedelta(hours = n)])
                final_df = pd.concat([final_df, d1.head(2)])
                df_SA_SM.drop(df_SA_SM.index[:2], inplace=True)
            else:
                df_SA_SM.drop(df_SA_SM.index[:2], inplace=True) #if, based on the process flow, that rows of SA_SERVICE MANAGEMENT do not need to be moved in the final event log, theya are only dropped by the related subprocess' log

        else:
            df_CC_CM.drop(df_CC_CM.loc[df_CC_CM['case']== i].index, inplace=True) #if, based on the process flow, that rows of SA_SERVICE MANAGEMENT do not need to be moved in the final event log, theya are only dropped by the related subprocess' log

        #FA_MANAGE_INSTALLATION
        if(((final_df['event'] == 'solicitation from the call center') & (final_df['case'] == i)).any() == True):
            if(((final_df['event'] == 'end_event_keep calm') & (final_df['case'] == i)).any() == False):
                if(((df_FA_MI['event'] == 'message_receive solicitation manage installation') & (df_FA_MI['case'] == i)).any() == True):
                    ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
                    d1 = pd.DataFrame()
                    d1 = df_FA_MI.head(2)
                    n = np.random.randint(1, 24)
                    d1.insert(2,"time",[ts, ts + timedelta(hours = n)])
                    final_df = pd.concat([final_df, d1.head(2)])
                    df_FA_MI.drop(df_FA_MI.index[:2], inplace=True)
                else:
                    ts = final_df.iloc[-1]['time'] + timedelta(minutes = 10)
                    new1 = [i, "message_receive solicitation manage installation", ts] # since there is no corresponce among the event logs that have been separetely simulated, it can happen that in the FA_MANAGE_INSTALLATION log there are no the tuple 
                                                                                       # related to the management of a complaint arrived by the service agent, in that scenario the tuples need to be manually created
                    new2 = [i, "reschedule appointments", ts + timedelta(hours = n)]   
                    final_df = final_df.append(pd.Series(new1, index=final_df.columns[:len(new1)]), ignore_index=True)
                    final_df = final_df.append(pd.Series(new2, index=final_df.columns[:len(new2)]), ignore_index=True)
        
        elif((((df_FA_MI['event'] == 'message_receive solicitation manage installation') & (df_FA_MI['case'] == i)).any() == True)):
            df_FA_MI.drop(df_FA_MI.index[:2], inplace=True)
        
        
        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_FA_MI.head(4)
        n = np.random.randint(24, 120)
        fuel =np.random.randint(1,7) # the attribute fuelUsed is randomly created
        d1.insert(2,"time",[ts + timedelta(hours = n), ts + timedelta(hours = n+0.5),ts + timedelta(hours = n+0.5) , ts + timedelta(hours = n+0.6)])
        d1.insert(3,"fuelUsed",[fuel,'','',''])
        final_df = pd.concat([final_df, d1.head(4)])
        df_FA_MI.drop(df_FA_MI.index[:4], inplace=True)

        #SA_SERVICE_MANAGEMENT
        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_SA_SM.head(3)
        d1.insert(2,"time",[ts + timedelta(minutes = 1), ts + timedelta(minutes = 2), ts + timedelta(minutes = 3)])
        d1.insert(3,"status",['in full','','']) #injection of the right value for the attribute 'status'
        final_df = pd.concat([final_df, d1.head(3)])
        df_SA_SM.drop(df_SA_SM.index[:3], inplace=True)

        #CC_ORDER_MANAGEMENT
        ts = final_df.iloc[-1]['time'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_CC_OM.head(2)
        d1.insert(2,"time",[ts + timedelta(minutes=1), ts + timedelta(minutes = 2)])
        final_df = pd.concat([final_df, d1.head(2)])
        df_CC_OM.drop(df_CC_OM.index[:2], inplace=True)

        #CC_COMPLIANT_MANAGEMENT
        if(((final_df['event'] == 'solicitation from the call center') & (final_df['case'] == i)).any() == True):
            ts = final_df.iloc[-1]['time'] + timedelta(minutes = i)
            d1 = pd.DataFrame()
            d1 = df_CC_CM.head(2)
            d1.insert(2,"time",[ts + timedelta(minutes=1), ts + timedelta(minutes = 2)])
            final_df = pd.concat([final_df, d1.head(2)])
            df_CC_CM.drop(df_CC_CM.index[:2], inplace=True)

        #SA_PAYMENT_MANAGEMENT
        d1 = pd.DataFrame()
        d1 = df_SA_PM.loc[df_SA_PM['case'] == i]
        lunghezza = d1.shape[0] # differently to the other subprocesses, for payment management it is not possible to know a-priori the number of events, for this reason all the tuples of SA_PAYMENT_MANAGEMENT related
                                # to that case are taken simoultaneously and then their attributes are handled using lists
        lista = []
        terms = []
        pt = np.random.randint(3,14) # the attribute 'paymentTerms' is randomly created
        receipt = np.random.randint(1,pt-1) # the moment in which the payment receipt is received is created randomly
        
        for t in range(lunghezza):
            if(df_SA_PM.iloc[t].event in ['event_TIMER_payment terms expire']):
                lista.append(ts+timedelta(days = pt))
                ts = ts+timedelta(days = pt)
            elif(df_SA_PM.iloc[t].event in ['event_MESSAGE_payment receipt']):
                 lista.append(ts+timedelta(days = receipt)) 
            else:
                  lista.append(ts+timedelta(minutes = receipt)) 
            if(df_SA_PM.iloc[t].event in ['event_MESSAGE_send invoice', 'event_MESSAGE_send solicitation with new amount']):
                terms.append(pt)
            else:
                terms.append('')
              
        d1.insert(2,"time", lista)
        d1.insert(3,"paymentTerms", terms)
        final_df = pd.concat([final_df, d1.head(lunghezza)])
        df_SA_PM.drop(df_SA_PM.loc[df_SA_PM['case']== i].index, inplace=True)
        

    else: # it is the case of an unavailable order
        
        #CC_ORDER_MANAGEMENT
        ts = onlyTS.iloc[0]['completeTime'] + timedelta(minutes = j)
        d1 = pd.DataFrame()
        d1 = df_CC_OM.head(4)
        d1.insert(2,"time", [ts, ts+ timedelta(minutes = 1), ts +timedelta(minutes = 10), ts +timedelta(minutes = 11)])
        final_df = pd.concat([final_df, d1.head(4)])
        df_CC_OM.drop(df_CC_OM.index[:4], inplace=True)
        df_CC_CM.drop(df_CC_CM.loc[df_CC_CM['case']== i].index, inplace=True)
        df_FA_MI.drop(df_FA_MI.loc[df_FA_MI['case']== i].index, inplace=True)
        df_SA_PM.drop(df_SA_PM.loc[df_SA_PM['case']== i].index, inplace=True)
        df_SA_SM.drop(df_SA_SM.loc[df_SA_SM['case']== i].index, inplace=True) 
    

In [ ]:
#Exporting of the final event logs
final_df.set_index('case', inplace = True)
final_df.to_csv('finalEventLog.csv')